# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2021-s109a/blob/master/lectures/crest.png?raw=true"> CS-S109A Introduction to Data Science 

## Homework 4:  Logistic Regression and PCA

**Harvard University**<br/>
**Summer 2021**<br/>
**Instructors**: Kevin Rader


<hr style='height:2px'>

---



In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

### INSTRUCTIONS

- To submit your assignment follow the instructions given in Canvas.
- Restart the kernel and run the whole notebook again before you submit. 
- If you submit individually and you have worked with someone, please include the name of your [one] partner below. 
- As much as possible, try and stick to the hints and functions we import at the top of the homework, as those are the ideas and tools the class supports and is aiming to teach. And if a problem specifies a particular library you're required to use that library, and possibly others from the import list.
- Please use .head() when viewing data. Do not submit a notebook that is excessively long because output was not suppressed or otherwise limited. 

**Partners: Sunil Kumar Shambulingaiah and Steven Devisch** 

In [2]:
import numpy as np
import pandas as pd
import sklearn as sk

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import zipfile


import seaborn as sns
sns.set()

# if you want to do a 2-sample t-test:
from scipy.stats import ttest_ind

<div class='theme'> Cancer Classification from Gene Expressions </div>

In this problem, we will build a classification model to distinguish between two related classes of cancer, acute lymphoblastic leukemia (ALL) and acute myeloid leukemia (AML), using gene expression measurements. The dataset is provided in the file `data/genomic_data.csv`. Each row in this file corresponds to a tumor tissue sample from a patient with one of the two forms of Leukemia. The first column contains the cancer type, with **0 indicating the ALL** class and **1 indicating the AML** class. Columns 2-7130 contain expression levels of 7129 genes recorded from each tissue sample. 

In the following questions, we will use linear and logistic regression to build classification models for this data set. 


<div class='exercise'><b> Question 1 [15 pts]: Data Exploration </b></div>

The first step is to split the observations into an approximate 75-25 train-test split.  Below is some code to do this for you (we want to make sure everyone has the same splits). It also prints the dataset's shape before splitting and after splitting. `Cancer_type` is our target column.


**1.1** Take a peek at your training set: you should notice the severe differences in the measurements from one gene to the next (some are negative, some hover around zero, and some are well into the thousands). To account for these differences in scale and variability, normalize each predictor to vary between 0 and 1. **NOTE: for the entirety of this homework assignment, you will use these normalized values, not the original, raw values**. Normalizing genomic data is a fairly standard first step.


**1.2** The training set contains more predictors than observations. What problem(s) can this lead to in fitting a classification model to such a dataset? Explain in 3 or fewer sentences.


**1.3** Determine which single gene individually discriminates between the two cancer classes the best (consider every gene in the dataset) and call it `best_predictor`.

Plot two histograms of your `best_predictor` -- one using the training set and another using the testing set. The histogram should clearly distinguish two different `Cancer_type` classes.

**Hint:** You may use any reasonable approach to determine the `best_predictor`, but please use something very simple (whether taught in this class or elsewhere).


**1.4** Using `best_predictor`, create a classification model by simply eye-balling a value for this gene that would discriminate the two classes the best (do not use an algorithm to determine for you the optimal coefficient or threshold; we are asking you to provide a rough estimate / model by manual inspection). Justify your choice in 1-2 sentences. Report the accuracy of your hand-chosen model on the test set.

<hr> 


### Answers

**The first step is to split the observations into an approximate 75-25 train-test split. Below is some code to do this for you (we want to make sure everyone has the same splits). Print dataset shape before splitting and after splitting. `Cancer_type` is our target column.**

In [3]:
np.random.seed(10)
df = pd.read_csv('data/genomic_data.csv', index_col=0)
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'Cancer_type'], 
                                                         df.Cancer_type, test_size=0.25, 
                                                         random_state = 109, 
                                                         stratify = df.Cancer_type)

In [4]:
print(df.shape)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(df.Cancer_type.value_counts(normalize=True))

(752, 7130)
(564, 7129) (188, 7129) (564,) (188,)
0.0    0.511968
1.0    0.488032
Name: Cancer_type, dtype: float64


**Comment:** there seem to be more columns than rows. The dataset does seem to be balanced, 

**1.1 Take a peek at your training set: you should notice the severe differences in the measurements from one gene to the next (some are negative, some hover around zero, and some are well into the thousands). To account for these differences in scale and variability, normalize each predictor to vary between 0 and 1. **NOTE: for the entirety of this homework assignment, you will use these normalized values, not the original, raw values.**


In [ ]:
X_train.describe()

In [ ]:
# archive column names as scale_transformer produces an array, removing column names
columns = X_train.columns
# scale the datasets
# use minmaxscalar as some predictors are negative
scale_transformer = MinMaxScaler(copy=True).fit(X_train)
X_train = pd.DataFrame(scale_transformer.transform(X_train))
X_test = pd.DataFrame(scale_transformer.transform(X_test))

# reapply columns
X_train.columns = columns
X_test.columns = columns

print(X_train.shape)
X_train.head()

**1.2 The training set contains more predictors than observations. What problem(s) can this lead to in fitting a classification model to such a dataset? Explain in 3 or fewer sentences.**

It is likely the predictors are correlated. The parameters are unidentifiable. Predictions on the training set will be perfect, but not on the test set.

**1.3 Determine which single gene individually discriminates between the two cancer classes the best (consider every gene in the dataset) and call it `best_predictor`.**

**Plot two histograms of your `best_predictor` -- one using the training set and another using the testing set. The histogram should clearly distinguish two different `Cancer_type` classes.**

In [ ]:
best_accuracy = -1
best_model = None

predictors = X_train.columns
for predictor in predictors:
    
    logref = LogisticRegression(solver='liblinear', max_iter=1000, C=100000)
    logref.fit(X_train[predictor].values.reshape(-1, 1), y_train)
    y_hat_test = logref.predict(X_test[predictor].values.reshape(-1, 1))
    cur_accuracy = accuracy_score(y_test.to_numpy(), y_hat_test)

    if cur_accuracy > best_accuracy:
        best_accuracy = cur_accuracy
        best_model = logref
        best_predictor = predictor

In [ ]:
print("Best found logistic regression model:", best_model,"\nAccuracy score:", best_accuracy)

In [ ]:
print("The best found logistic regression model uses this variable:", best_predictor)

In [ ]:
# 2 graphs side by side
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# historgram on training set 
ax[0].hist(X_train[best_predictor][y_train.values==1], alpha=0.5, label = 'AML')
ax[0].hist(X_train[best_predictor][y_train.values==0], alpha=0.5, label = 'ALL')
ax[0].set_xlabel('Standardized predictor ' +  best_predictor)
ax[0].set_ylabel('Number of cancer cases')
ax[0].set_title('AML and ALL distribution in train set by values of predictor ' +  best_predictor)
ax[0].legend()

ax[1].hist(X_test[best_predictor][y_test.values==1], alpha=0.5, label = 'AML')
ax[1].hist(X_test[best_predictor][y_test.values==0], alpha=0.5, label = 'ALL')
ax[1].set_xlabel('Standardized predictor ' +  best_predictor)
ax[1].set_ylabel('Number of cancer cases')
ax[1].set_title('AML and ALL distribution in test set by values of predictor ' +  best_predictor)
ax[1].legend()

fig.tight_layout(pad=3.0)
fig.suptitle("Histogram comparions between train and test set for best predictor: " + best_predictor, fontsize=12)

**Comment**: The histograms for bot the train and test set demonstrat how the best predictor correlates more with ALL at higher values of the predictor, and more with AML at lower values.

**1.4 Using `best_predictor`, create a classification model by simply eye-balling a value for this gene that would discriminate the two classes the best (do not use an algorithm to determine for you the optimal coefficient or threshold; we are asking you to provide a rough estimate / model by manual inspection). Justify your choice in 1-2 sentences. Report the accuracy of your hand-chosen model on the test set.**


In [ ]:

# default to ALL
y_hat_test[:] = 0

# set threshold for ALL above 0.5
thresh = 0.5
AML_indices = X_test[best_predictor].values > thresh
# set to AML if predictor exceeds the threshold value
y_hat_test[AML_indices] = 1

#@Sunil something's wrong in this calculation. Accuracy should be closer to 66% (see below)
eyeball_accuracy = accuracy_score(y_hat_test, y_test.to_numpy())
print("The eye-ball model has an accuracy of: " + str(eyeball_accuracy) + " on the test set")

---

<div class='exercise'> <b> Question 2 [35pts]: Logistic Regression Modeling </b> </div>


**2.1** Fit a simple logistic regression model to the training set using the single gene predictor `best_predictor` to predict cancer type.  Carefully interpret the coefficient estimates for this model.

*Remember, you need to set the regularization parameter for sklearn's logistic regression function to be a very large value in order to **not** regularize (use `C=100000` or `penalty = "none"`).

**2.2** Plot the logistic curves for the model in 2.1 ($y$-axis is probability scale, $x$-axis is `best_predictor`).  Interpret this plot: at what values of your `best_predictor` will you predict the patient to have ALL?  How does this compare to your eeballed value from 1.4?

**2.3** Calculate the training and test classification accuracies of this model in 2.1. How do these compare to the eye-balled model from 1.4?


**2.4** Next, fit a multiple logistic regression model with **all** the gene predictors from the data set (reminder: for this assignment, we are always using the normalized values). How does the classification accuracy of this model compare with the models fitted with a single gene (on both the training and test sets)?  

**2.5** Print out and interpret the logistic regression coefficients for  `best_predictor` from both the simple logistic and multiple logistic regression models from the previous two parts.  Do they agree or disagree?  What does this indicate?

**2.6** Now let's use regularization to improve the predictions from the multiple logistic regression model. Specifically, use LASSO-like regularization and 5-fold cross-validation to fit the model on the training set (choose between 20 reasonable values of $\lambda$). Report the classification accuracy on both the training and testing set.

**2.7** How many predictors are considered as important features in this regularized model?  What does that say about the full logistic regression model in problem 2.4?


## Answers

**2.1 Fit a simple logistic regression model to the training set using the single gene predictor `best_predictor` to predict cancer type. Carefully interpret the coefficient estimates for this model.**

In [ ]:
logreg = LogisticRegression(solver='liblinear', max_iter=1000, C=100000)
logreg.fit(X_train[best_predictor].values.reshape(-1, 1), y_train)
y_hat_test = logreg.predict(X_test[best_predictor].values.reshape(-1, 1))
cur_accuracy = accuracy_score(y_test.to_numpy(), y_hat_test)
print("Accuracy for a model with 'best' predictor " + best_predictor + " is: " + str(cur_accuracy))
print(logref.coef_)

In [ ]:
# convert log odds into odds for the coeffinet associated with the best predictor
odds = np.exp(logreg.coef_[0])

**Coefficient interpretation:** For every one-unit increase in the best predictor 'Y08612_at', the odds that the observation is in AML are 0.00233 times as large as the odds that the observation is not in AML when all other variables are held constant.

**2.2 Plot the logistic curves for the model in 2.1 ($y$-axis is probability scale, $x$-axis is `best_predictor`).  Interpret this plot: at what values of your `best_predictor` will you predict the patient to have ALL?  How does this compare to your eeballed value from 1.4?**


In [ ]:
# seaborn provides a convenient method to plotting logistic curves
sbplt = sns.regplot(x=X_train[best_predictor], y=y_train.values, logistic=True)
# add a horizontal line at the cut-off point
sbplt.axhline(0.5, c='red', alpha=0.5)
sbplt.set(xlabel='Best predictor: ' + best_predictor, ylabel='Probability', title='Logistic curve for the best predictor model')

**Comment**: We observe that the prediction flips at a Y08612_at value of a little bit below 0.5. This is similar to our ealier guess. At values below 0.5 for standardized Y08612_at, we predict a patient to have ALL.

**2.3 Calculate the training and test classification accuracies of this model in 2.1.  How do these compare to the eye-balled model from 1.4?**

In [ ]:
# calculate training accuracies
y_hat_train = logreg.predict(X_train[best_predictor].values.reshape(-1, 1))
train_accuracy = accuracy_score(y_train.to_numpy(), y_hat_train)
print("Accuracy on the training set is:", train_accuracy)

# calculate test accuracies
y_hat_test = logreg.predict(X_test[best_predictor].values.reshape(-1, 1))
test_accuracy = accuracy_score(y_test.to_numpy(), y_hat_test)
print("Accuracy on the test set is:", test_accuracy)

**Comment:** The eyeballed model and the logistic regression model perform similarly. This seems reasonable. 


**2.4 Next, fit a multiple logistic regression model with *all* the gene predictors from the data set (reminder: for this assignment, we are always using the normalized values). How does the classification accuracy of this model compare with the models fitted with a single gene (on both the training and test sets)?**


In [ ]:
# fit a model with all predictors (without regularizing)
logreg_all = LogisticRegression(solver='liblinear', max_iter=1000, C=100000)
logreg_all.fit(X_train, y_train)

# calculate training accuracies
y_hat_train = logreg_all.predict(X_train)
train_accuracy = accuracy_score(y_train.to_numpy(), y_hat_train)
print("Accuracy on the training set is:", train_accuracy)

# calculate test accuracies
y_hat_test_all = logreg_all.predict(X_test)
test_accuracy = accuracy_score(y_test.to_numpy(), y_hat_test_all)
print("Accuracy on the test set is:", test_accuracy)


**Comment:** As anticipated above since there are more predictors than observations, model accuracy for the training set becomes 100%. Model accuracy for the test set has increased as well.

**2.5 Print out and interpret the logistic regression coefficients for `best_predictor` from both the simple logistic and multiple logistic regression models from the previous two parts.  Do they agree or disagree?  What does this indicate?**

In [ ]:
logreg_all.coef_

In [ ]:
# store the position of the best predictor
best_pred_pos = X_train.columns.get_loc(best_predictor)

# coef_list = np.ndarray.tolist(logreg_all.coef_)
# coef_list[0][best_pred_pos]
print('The coefficient for best_predictor in simple logistic regression is:', logref.coef_[0][0])
print('The coefficient for best_predictor in multiple logistic regression is:', logreg_all.coef_[0][best_pred_pos])



**Comment:** The coefficients for simple and multiple logistic regression are very different. This is likely due to high collinearity between the different variables.

**2.6 Now let's use regularization to improve the predictions from the multiple logistic regression model. Specifically, use LASSO-like regularization and 5-fold cross-validation to fit the model on the training set (choose between 20 reasonable values of $\lambda$). Report the classification accuracy on both the training and testing set.**

In [ ]:
#Your answer
best_accuracy = -1
best_model = None
accuracies = []

# experiment with different values of different order of magnitudes
cs_vals = np.ndarray.tolist((np.ones(20)*10)**np.arange(-10,10,1).astype(int))
# overriding cs_vals as only integers work
cs_vals = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100,1000]
# @ Sunil. subseting cvals to lower number as 20 take long to compute. Put back befor submitting.
cs_vals = [4,5,6]

# adding a solver choice in case we also want to experiment with different solvers
solvers = ['liblinear']
for c in cs_vals:
    for sol in solvers:
        # use l1 for lasso and 5 cross validation (cv) folds
        logref =  LogisticRegressionCV(Cs=c, solver=sol, penalty='l1', cv=5)
        logref.fit(X_train, y_train)
        y_hat_test = logref.predict(X_test)
        cur_accuracy = accuracy_score(y_test.to_numpy(), y_hat_test)
        
        # adding accuracy to a list in case we want to show how accuracy changes with lambda
        accuracies.append(cur_accuracy)
        if cur_accuracy > best_accuracy:
            best_accuracy = cur_accuracy
            best_reg_model = logref
            best_lambda = c
            
# predictions
y_train_pred = best_reg_model.predict(X_train) 
y_test_pred = best_reg_model.predict(X_test)

# accuracy
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)

In [ ]:
print("The best lambda was:", best_lambda)
print("Errors on the training set were:", train_score)
print("Errors on the test set were:",test_score)

**Comment**: after regularization, test scores improved significantly

**2.7 How many predictors are considered as important features in this regularized model?  What does that say about the full logistic regression model in problem 2.4?**


In [ ]:
# calculate the number of non-zero coeffients in the model
print("The number of non-zero coefficients in the model is:", sum(x > 0 for x in best_reg_model.coef_[0]))

**Comment** The regularized model retained nineteen predictors. We can conclude that the initial predictor set - as used in the full logistic regression model - was highly correlated.

---

<div class='exercise'> <b> Question 3 [10pts]: $k$-NN Classification </b> </div>

**3.1** Use 5-fold cross-validation to select $k$ for a $k$-NN classification model based on the full predictor set.  Choose between `ks = [1,3,5,7,10,15,20,50,100]`. 

**3.2** Provide the confusion matrix for 3 models: (i) the full multiple logistic regression model from 2.4, (ii) the best regularized model from 2.6, and (iii) the best $k$-NN from the previous part. Report the false positive and false negative rates (all in the test set).  Briefly interpret what you notice.


### Answers

**3.1 Use 5-fold cross-validation to select $k$ for a $k$-NN classification model based on the full predictor set.  Choose between `ks = [1,3,5,7,10,15,20,50,100]`.  Report your chosen $k$, and report the misclassification rate on both the train and test sets for the model using your chosen $k$.**

In [ ]:
ks = [1,3,5,7,10,15,20,50,100]
cv_folds = 5
best_accuracy = -1
acc = []

for k in ks:
    knn_model = KNeighborsClassifier(k)
    cur_accuracy = np.mean(cross_val_score(knn_model, X_train, y_train, cv=cv_folds))
    acc.append(cur_accuracy)
    if cur_accuracy > best_accuracy:
        best_accuracy = cur_accuracy
        best_knn_model = knn_model 
        best_k = k

In [ ]:
plt.figure(figsize = (8,6))    
plt.plot(ks, acc)
plt.xlabel('k value')
plt.ylabel('Classification accuracy')

In [ ]:
print("The best value for k is: ", best_k)
print("Classification accuracy for this k:", best_accuracy)

**3.2 Provide the confusion matrix for 3 models: (i) the full multiple logistic regression model from 2.4, (ii) the best regularized model from 2.6, and (iii) the best $k$-NN from the previous part. what are the  false positive and false negative rates in these 3 models (all in the test set)?  Briefly interpret what you notice.**

In [ ]:
# helper function to calculate false positive and false positive rates
def calc_confusion_rate(confusion_matrix):
        TP = confusion_matrix.loc['true:0','pred:0']
        TN =  confusion_matrix.loc['true:1','pred:1']        
        FP =  confusion_matrix.loc['true:1','pred:0']
        FN =  confusion_matrix.loc['true:0','pred:1']
        FP_rate = FP / (FP+TN)
        FN_rate = FN / (FN+TP)
        return FP_rate, FN_rate

In [ ]:
# set the set of models and rates we seek to explore
models = [logreg_all, best_reg_model, best_knn_model]
model_descriptions = ['Full_logistic', 'Regularized_bLogistic', 'KNN']
confusion_rates = ['False_Positive_Rate', 'False_Negative_Rate']

In [ ]:
# prepare a dataframe to represent rates for each model
confusion_rate_df = pd.DataFrame(
    index=model_descriptions,
    columns=['False_Positive_Rate', 'False_Negative_Rate']
)

In [ ]:
index = 0
for model in models:
    # confusion matrix for model with all predictors
    model.fit(X_train,y_train)
    y_hat_test = model.predict(X_test)
    conf_df = pd.DataFrame(
        metrics.confusion_matrix(y_test, y_hat_test), 
        index=['true:0', 'true:1'], 
        columns=['pred:0', 'pred:1'])
    curr_model_descr = model_descriptions[index]
    print("\n Confusion matrix for:", curr_model_descr,"\n")

    # calculate false positive and false negative rate
    FP_rate, FN_rate = calc_confusion_rate(conf_df)
    
    # Add rates to dataframe for clear comparison 
    confusion_rate_df.loc[curr_model_descr,'False_Positive_Rate'] = FP_rate
    confusion_rate_df.loc[curr_model_descr,'False_Negative_Rate'] = FN_rate
    print(conf_df)
    index = index + 1

In [ ]:
# display the rates by model in percentage format
confusion_rate_df.style.format({
    'False_Positive_Rate': '{:,.1%}'.format,
    'False_Negative_Rate': '{:,.1%}'.format,
})

**Comment** Both false positive and false negative rate are lowest for regularized logistic regression model. It is therefore reasonable to conclude that the regularized logistic regression model is the best model. K nearest neighbor also performs better than the full logistic regression model.

---

#### <div class='exercise'><b> Question 4 [15 pts]: Performing Principal Components Analysis </b></div>

**4.1** Create the full PCA decomposition of `X_train` and apply the transformation to both `X_train` and `X_test`.  Report the shape of both of these.  What is the limiting factor for the maximum number of PCA components for this data set? 

*Hint: be sure to standardize before performing PCA.

**4.2** PCA is often solely used to help in visualizing high-dimensional problems.  Plot the scatterplot of the second PCA vector of train on the $Y$-axis and the first PCA vector of train on the $X$-axis (be sure to denote the classes via different colors and markings).  In 2-3 sentences, explain why using the scatterplot of the top 2 PCA vectors is a useful approach to visualize a high dimensional classification problem.

**4.3** Determine and report the variance explained in `X_train` based on the top 2 PCA vectors.  Determine and report how many PCA vectors are needed so that 90\% of the variability in the predictors is explained, and create a plot to illustrate this result (Hint: look at cumulative explained variability vs. number of PCA components used).  Select a reasonable value for the number of components that balances representativeness (of the predictors) with parsimony. 



### Answers

**4.1 Create the full PCA decomposition of X_train and apply the transformation to both X_train and X_test. Report the shape of both of these. What is the limiting factor for the maximum number of PCA components for this data set?**

In [ ]:
# create/fit the 'full' pca transformation for X_train
pca_train = PCA().fit(X_train)

# apply the pca transformation to the full predictor set
pcaX_train = pca_train.transform(X_train)

# convert to a data frame
pcaX_df_train = pd.DataFrame(pcaX_train)

print("Original train dataset shape:",pca_train.components_.shape);
print("Shape PCA transformed X_train:", pcaX_train.shape)


In [ ]:
# create/fit the 'full' pca transformation for X_test
pca_test = PCA().fit(X_test)

# apply the pca transformation to the full predictor set
pcaX_test = pca_test.transform(X_test)

# convert to a data frame
pcaX_df_test = pd.DataFrame(pcaX_test)

print("Original test dataset shape:",pca_test.components_.shape);
print("Shape PCA transformed X_test:", pcaX_test.shape)


@Sunil. I'm not sure if I'm answering the question.
**Comment"** If the number of observations 𝑛 is less than or equal to the number of features, the 𝑛-th Principal Component (PC) will be constant zero (eigenvalue = 0). The number of non-trivial PCs is 𝑛−1.


**4.2 PCA is often solely used to help in visualizing high-dimensional problems. Plot the scatterplot of the second PCA vector on the  𝑌 -axis and the first PCA vector on the  𝑋 -axis (be sure to denote the classes via different color/markings). In 2-3 sentences, explain why using the scatterplot of the top 2 PCA vectors is a useful approach to visualize a high dimensional classification problem.**

In [ ]:
# fix the indexes. @Sunil I don't understand why I have to do this
# without this, I get an error
pcaX_df_train.index = y_train.index

In [ ]:
# Plot the response over the first 2 PCA component vectors
plt.scatter(pcaX_df_train.iloc[:, 0][y_train==0],pcaX_df_train.iloc[:, 1][y_train==0])
plt.scatter(pcaX_df_train.iloc[:, 0][y_train==1],pcaX_df_train.iloc[:, 1][y_train==1])

plt.legend(["ALL","AML"])
plt.xlabel("First PCA Component Vector (Z1)")
plt.ylabel("Second PCA Component Vector (Z2)")
plt.title("ALL and AML as a function of top 2 principal components")

@Sunil. I suspect this explanation could be improved.
**Comment** Most useful visualizations are limited to two dimensions. Visualizing the effect of a high number of predictors is therefore challenging. PCA allows to summarize the effect of multiple predictors in a a summarized 2-dimensional space.

**4.3 Determine and report the variance explained in `X_train` based on the top 2 PCA vectors.  Determine and report how many PCA vectors are needed so that 90\% of the variability in the predictors is explained, and create a plot to illustrate this result (Hint: look at cumulative explained variability vs. number of PCA components used).  Select a reasonable value for the number of components that balances representativeness (of the predictors) with parsimony.**

In [ ]:
# variance explained by fist two components
print("Variance explained by the first principal component:",pca_train.explained_variance_ratio_[0])
print("Variance explained by the second principal component:",pca_train.explained_variance_ratio_[1])


In [ ]:
target_variability_explained = .9
cum_PCA = np.cumsum(pca_train.explained_variance_ratio_)

# asses at which point we reach a certain percentage for variability explained
target_variability_at = len(cum_PCA) - sum(cum_PCA > target_variability_explained)

print('Number of principal components needed to explain', 
      target_variability_explained, "% of variability:",
      target_variability_at )

In [ ]:
# plot the cumulative "variance explained" curve for X_train
plt.plot(np.cumsum(pca_train.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.suptitle('number of principal componenets needed for 90% variance ')
plt.axhline(0.9, c='red', alpha=0.5)


**Comment** A reasonable number of components seems to be any number between 75 and 400. For parsimony reasons, I would recommend 75, roughly the point where the curve starts flattening.

---

<div class='exercise'><b> Question 5 [10 pts]: Principal Components Regression (PCR) </b></div>

**5.1** Fit three separate Logistic Regression models using principal components as the predictors: (1) with just the first 2 PCA vectors, (2) with the number of component vectors you chose from 5.4 above, and (3) with the number of components that explain at least 90% of the variability in the predictor set. How do the classification accuracy values on both the training and test sets compare with these models?

**5.2** Use cross-validation to determine the best number of principal components. Try out the 3 values from the previous sub-part and optionally include other values as well. For the best performing model according to cross-validation, interpret what the model says about the relationship between `best_predictor` and `Cancer_type`.



### Answers

**5.1 Fit three separate Logistic Regression models using principal components as the predictors: (1) with just the first 2 PCA vectors, (2) with the number of component vectors you chose from 5.4 above, and (3) with the number of components that explain at least 90% of the variability in the predictor set. How do the classification accuracy values on both the training and test sets compare with these models?**

In [ ]:
# set the set of models and rates we seek to explore
pca_cnts = [2, 103, 75]
model_descriptions = ['2_PCA_vectors', 'Ninety_perc_pca_vectors (103)', 'Chosen_pca_vectors (75)']

best_accuracy = -1
best_pca_model = None
accuracies = []

In [ ]:
# prepare a dataframe to represent accuracy for each model
pca_cnt_acc_df = pd.DataFrame(
    index=model_descriptions,
    columns=['Train_accuracy', 'Test_accuracy']
)

In [ ]:
iteration_counter = 0
for pca_cnts in pca_cnts:
    # setting c high to avoid additional regularizatoin
    logreg =  LogisticRegression(C=1000000, solver='liblinear', max_iter=1000)
    # note we're using the pcaX dataset, subsetted to include the specific feature nr.
    logreg.fit(pcaX_df_train.iloc[:,:pca_cnts], y_train)

    
    y_hat_train = logreg.predict(pcaX_df_train.iloc[:,:pca_cnts])
    train_accuracy = accuracy_score(y_train.to_numpy(), y_hat_train)

    y_hat_test = logreg.predict(pcaX_df_test.iloc[:,:pca_cnts])
    test_accuracy = accuracy_score(y_test.to_numpy(), y_hat_test)

    # adding accuracy to a list in case we want to show how accuracy changes with pca_cnts
    accuracies.append(cur_accuracy)
    if cur_accuracy > best_accuracy:
        best_accuracy = cur_accuracy
        best_pca_model = logreg
        best_pca_cnt = pca_cnts
    
    # Add rates to dataframe for clear comparison 
    curr_model_descr = model_descriptions[iteration_counter]
    pca_cnt_acc_df.loc[curr_model_descr,'Train_accuracy'] = train_accuracy
    pca_cnt_acc_df.loc[curr_model_descr,'Test_accuracy'] = test_accuracy
    iteration_counter = iteration_counter + 1


In [ ]:
# display the rates by model in percentage format
print("Comparing different regression models by varying the number of principal components")
pca_cnt_acc_df.style.format({
    'Train_accuracy': '{:,.1%}'.format,
    'Test_accuracy': '{:,.1%}'.format,
})

**Comment** The 2-component model performs nearly as well as the chosen pca model. Since the two-component model is significantly more parsimoneous, it seems reasonable to prefer the two-component model.

**5.2 Use cross-validation to determine the best number of principal components. Try out the 3 values from the previous sub-part and optionally include other values as well. For the best performing model according to cross-validation, interpret what the model says about the relationship between your `best_predictor` and `Cancer_type`**.

In [ ]:
pca_cnts = [2, 10, 20, 30, 40, 50, 60, 75, 103, 175]
cs_vals = [4,5,6]# set the set of models and rates we seek to explore
model_descriptions =[]
cv_folds = 5

best_accuracy = -1
best_pca_cv_model = None
accuracies = []

In [ ]:
# prepare a dataframe to represent accuracy for each model

for pca_cnt in pca_cnts:
    model_description = str(pca_cnt) + "_PCA"
    model_descriptions.append(model_description)

pca_cnt_cv_acc_df = pd.DataFrame(
    index=model_descriptions,
    columns=['Train_accuracy', 'Test_accuracy'])

In [ ]:
iteration_counter = 0
for pca_cnts in pca_cnts:
    # adding cross validation and setting c high to avoid additional regularizatoin
    logreg =  LogisticRegressionCV(Cs=cs_vals, solver='liblinear', max_iter=1000, cv=cv_folds)
    # note we're using the pcaX dataset, subsetted to include the specific feature nr.
    logreg.fit(pcaX_df_train.iloc[:,:pca_cnts], y_train)
    
    y_hat_train = logreg.predict(pcaX_df_train.iloc[:,:pca_cnts])
    train_accuracy = accuracy_score(y_train.to_numpy(), y_hat_train)

    y_hat_test = logreg.predict(pcaX_df_test.iloc[:,:pca_cnts])
    test_accuracy = accuracy_score(y_test.to_numpy(), y_hat_test)

    # adding accuracy to a list in case we want to show how accuracy changes with pca_cnts
    accuracies.append(cur_accuracy)
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_pca_model = logreg
        best_pca_cnt = pca_cnts
    
    # Add rates to dataframe for clear comparison 
    curr_model_descr = model_descriptions[iteration_counter]
    pca_cnt_cv_acc_df.loc[curr_model_descr,'Train_accuracy'] = train_accuracy
    pca_cnt_cv_acc_df.loc[curr_model_descr,'Test_accuracy'] = test_accuracy
    iteration_counter = iteration_counter + 1


In [ ]:
# display the rates by model in percentage format
print("Comparing different regression models by varying the number of principal components with cross validation:")
pca_cnt_cv_acc_df.style.format({
    'Train_accuracy': '{:,.1%}'.format,
    'Test_accuracy': '{:,.1%}'.format,
})

In [ ]:
print("Best accuracy is",best_accuracy, "with", best_pca_cnt, "components")

**Interpret what the model says about the relationship between your best_predictor and Cancer_type**

In [ ]:
# Fit the PCR
scaler = sk.preprocessing.StandardScaler()
scaler.fit(X_train)
# @sunil not sure if we need this
# Z = scaler.transform(X_train)
pca = PCA(n_components=best_pca_cnt).fit(X_train)


In [ ]:
logit_pcr = []
pcrxz = []
iter = 0
# Run through each of the principle components, and grab the coeficients for the original predictors
for pca_cnt in range(best_pca_cnt):
    pca_cnt = pca_cnt+1
    logit_pcr.append(LogisticRegression(C=1000000,solver="lbfgs").fit(pcaX_df_train.iloc[:,:pca_cnt],y_train))
    pcrxz.append(logit_pcr[iter].coef_*np.transpose(pca.components_[0:1,:]).sum(axis=1))


# Add up components to produce the coefficent for best_predictor
resultsZ = np.vstack((pcrxz))
coef_sum = resultsZ.sum(axis=0)
print("The coefficient for the best predictor is:", coef_sum[best_pred_pos])
# convert log odds into odds for the coeffinet associated with the best predictor
odds = np.exp(coef_sum[best_pred_pos])
print("The odds for the best predictor are:",odds)


@Sunil. What do you think of this interpretation? This type of explanation is also used higher up.
**Comment**
Coefficient interpretation: For every one-unit increase in the best predictor 'Y08612_at', the odds that the observation is in AML are roughly equal to the odds that the observation is not in AML, when all other variables are held constant. 

<div class='exercise'><b> Question 6 [15 pts]: Evaluating Classifiers </b></div>

**6.1**: Another way to evaluate models in a classification setting is through an Area-under-the-ROC-Curve (AUC). Briefly explain what the AUC and the ROC are trying to do and how this approach differs from evaluating models based on misclassification rate (as you have done thus far in this problem set).

**6.2** Evaluate the 'best' models (best based on test misclassification: if there is a tie, choose the 'simplest' model) from each class of classification models using AUC.  That is calculate AUC for the following models:
- the best logistic regression model, whether regularized or not (question 2)
- the best $k$-NN model (question 3)
- the best PCR model (question 5)

For the model with the best AUC, plot the ROC. Briefly interpret your plot.

**6.3** Based on AUC, is there one clear stand-out winner or are a lot of models similar in prediction?  If you were to predict real cancer patients, how would use these models to predict cancer type?

*See extra information about ALL and AML at the bottom of this notebook.*

### Answers

**6.1 Another way to evaluate models in a classification setting is through an Area-under-the-ROC-Curve (AUC). Briefly explain what the AUC and the ROC are trying to do and how this approach differs from evaluating models based on misclassification rate (as you have done thus far in this problem set).**

The Receiver Operating Characteristics (FOC) curve illustrates the trade-off between True Positive Rate - how much correctly classified as 1 = and False Positive Rate (How much incorrectly classified as 1).

The ROC curve illustrates a range of possible cut off rates. A curve closer to the top left corner indicates a better classifier/Model.

AUC is the Area Under the Curve in the ROC plot. A higher AUC is better. AUC=1 means all positive examples come after your negative example. AUC = 0 means all negative examples come after your positive example. AUC=0.5 means a random classifier.

**6.2 use AUC to evaluate the 'best' models (best based on test misclassification: if there is a tie, choose the 'simplest' model) from each class of classification models.  That is calculate AUC for the following models:**
- the best logistic regression model, whether regularized or not (question 2)
- the best $k$-NN model (question 3)
- the best PCR model (question 5)

**For the model with the best AUC, plot the ROC. Briefly interpret your plot.**



In [ ]:
# prepare a dataframe to represent accuracy for each model
model_descriptions = ['Best logistic model', 'Best kNN model', 'Best PCA model']
auc_by_model_df = pd.DataFrame(
    index=model_descriptions,
    columns=['AUC'])

In [ ]:
# AUC Best logisic model
fpr, tpr, thresholds = metrics.roc_curve(y_test, best_reg_model.predict_proba(X_test)[:,1])
roc_auc = metrics.auc(fpr, tpr)
# add auc to table
auc_by_model_df.loc['Best logistic model','AUC'] = roc_auc

In [ ]:
# AUC Best Knn model
fpr, tpr, thresholds = metrics.roc_curve(y_test, best_knn_model.predict_proba(X_test)[:,1])
roc_auc = metrics.auc(fpr, tpr)
# add auc to table
auc_by_model_df.loc['Best kNN model','AUC'] = roc_auc

In [ ]:
# AUC Best PCA model
fpr, tpr, thresholds = metrics.roc_curve(y_test, best_pca_model.predict_proba(pcaX_df_test.iloc[:,:2])[:,1])
roc_auc = metrics.auc(fpr, tpr)
# add auc to table
auc_by_model_df.loc['Best PCA model','AUC'] = roc_auc

In [ ]:
# display the rates by model in percentage format
print("Comparing AUC's between different type of models:")
auc_by_model_df.style.format({
    'AUC': '{:,.1%}'.format,
})

In [ ]:
#####
#Make ROC curves to evaluate a model's overall useability.
#####

plt.rcParams['figure.dpi'] = 120
from sklearn.metrics import roc_curve, auc
import seaborn as sns

# a function to make 'pretty' ROC curves for this model
def make_roc(name, clf, ytest, xtest, ax=None, labe=5, proba=True, skip=0):
    initial=False
    if not ax:
        ax=plt.gca()
        initial=True
    if proba:#for stuff like logistic regression
        fpr, tpr, thresholds=roc_curve(ytest, clf.predict_proba(xtest)[:,1])
        
    roc_auc = auc(fpr, tpr)
    if skip:
        l=fpr.shape[0]
        ax.plot(fpr[0:l:skip], tpr[0:l:skip], '.-', alpha=0.3, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    else:
        ax.plot(fpr, tpr, '.-', alpha=0.3, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    label_kwargs = {}
    label_kwargs['bbox'] = dict(
        boxstyle='round,pad=0.3', alpha=0.2, mutation_scale =1
    )
    if labe!=None:
        for k in range(0, fpr.shape[0],labe):
            #from https://gist.github.com/podshumok/c1d1c9394335d86255b8
            threshold = str(np.round(thresholds[k], 2))
            ax.annotate(threshold, (fpr[k], tpr[k]), **label_kwargs)
    if initial:
        ax.plot([0, 1], [0, 1], 'k--')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05]) 
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title('ROC')
    ax.legend()
    return ax

In [ ]:
# @Sunil This doesn't look great, but the example from teh lab looks the same on my computer
make_roc("Logistic", best_reg_model, y_test, X_test, ax=None, labe=5, proba=True, skip=1);

**Comment** We can observe that the model is pretty good. The ROC curve goes up steeply. The model can discern most true positives before it starts identifying false positives.

**6.3 Based on AUC, is there one clear stand-out winner or are a lot of models similar in prediction?  If you were to predict real cancer patients, how would use these models to predict cancer type?**

**Comment** Based on AUC, there is no clear stand-out winner. The logistic regression and KNN models both have a similar and excellent AUC score (at 87%). At 83%, the PCA model performs only marginally worse.

Since all the models work relatively well, we could consider using multiple models to predict real cancer patients. If all three models predict the same cancer type, our confidence in the prediction would increase. Nevertheless, we should not feel too confident as false positives remain and the base prevalence for leuchemia is relatively low (~14 per 100,000). The prediction should be confirmed with a higher-confidence test, such as a biopsy. 


<hr style="height:2pt">


**Additional Information**

Acute Lymphoblastic Leukemia (ALL):
- About 98% of children with ALL go into remission within weeks after starting treatment.
- About 90% of those children can be cured. Patients are considered cured after 10 years in remission.

Acute Myeloid Leukemia (AML):
- In general, children with AML are seen as lower risk than adults. 
- Around 85 to 90 percent of children with AML will go into remission after induction, according to the American Cancer Society. AML will return in some cases.  
- The five-year-survival-rate for children with AML is 60 to 70 percent.

<hr style="height:2pt">